In [1]:
#Se importan las librerías necesarias para el problema
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import missingno as msng
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 10)

In [7]:
df = pd.read_csv('../data/df_procesado.csv')

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
import xgboost as xgb
from xgboost import plot_importance
from sklearn.metrics import roc_auc_score
from numpy import mean

In [9]:
data = shuffle(df[['OPERA', 'MES', 'periodo_dia','TIPOVUELO', 'SIGLADES', 'DIANOM', 'atraso_15']], random_state = 111)

In [10]:
data

,OPERA,MES,periodo_dia,TIPOVUELO,SIGLADES,DIANOM,atraso_15
13375,Sky Airline,3,mañana,N,Antofagasta,Viernes,0
35799,Grupo LATAM,7,noche,N,Concepcion,Jueves,0
48176,Grupo LATAM,9,noche,N,Arica,Miercoles,0
58675,Grupo LATAM,11,mañana,N,Iquique,Viernes,0
23374,Sky Airline,5,mañana,N,Valdivia,Lunes,0
...,...,...,...,...,...,...,...
37831,Grupo LATAM,7,tarde,I,Sao Paulo,Domingo,0
28790,Sky Airline,6,mañana,N,La Serena,Viernes,1
25257,Grupo LATAM,5,tarde,N,Temuco,Domingo,0
4820,Grupo LATAM,1,noche,I,Buenos Aires,Jueves,0


In [14]:
features = pd.concat([pd.get_dummies(data['OPERA'], prefix = 'OPERA'),pd.get_dummies(data['TIPOVUELO'], prefix = 'TIPOVUELO'),pd.get_dummies(data['periodo_dia'], prefix = 'periodo_dia'), pd.get_dummies(data['MES'], prefix = 'MES')], axis = 1)
label = data['atraso_15']

In [15]:
features_importantes = features[['MES_7', 'TIPOVUELO_I', 'OPERA_Copa Air', 'OPERA_Latin American Wings',
                                'MES_12', 'OPERA_Grupo LATAM', 'MES_10', 'OPERA_JetSmart SPA', 'OPERA_Air Canada',
                                'MES_9', 'OPERA_American Airlines']]
labels_ = label

In [11]:
# This script needs these libraries to be installed: 
#   numpy, sklearn

import wandb
from wandb.sklearn import plot_precision_recall, plot_feature_importances
from wandb.sklearn import plot_class_proportions, plot_learning_curve, plot_roc

import numpy as np
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


# load and process data
wbcd = datasets.load_breast_cancer()
feature_names = wbcd.feature_names
labels = wbcd.target_names

test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(wbcd.data, wbcd.target, test_size=test_size)

# train model
model = RandomForestClassifier()
model.fit(X_train, y_train)
model_params = model.get_params()

# get predictions
y_pred = model.predict(X_test)
y_probas = model.predict_proba(X_test)
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

# start a new wandb run and add your model hyperparameters
wandb.init(project='my-awesome-project', config=model_params)

# Add additional configs to wandb
wandb.config.update({"test_size" : test_size,
                    "train_len" : len(X_train),
                    "test_len" : len(X_test)})

# log additional visualisations to wandb
plot_class_proportions(y_train, y_test, labels)
plot_learning_curve(model, X_train, y_train)
plot_roc(y_test, y_probas, labels)
plot_precision_recall(y_test, y_probas, labels)
plot_feature_importances(model)

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

In [20]:
#import data
#df = data

# Separating the independent variables from dependent variables
X = features
y = label

# values to evaluate
over_values = [0.3,0.4,0.5]
under_values = [0.7,0.6,0.5]
for o in over_values:
  for u in under_values:
    # define pipeline
    model = xgb.XGBClassifier(random_state=1, learning_rate=0.01)
    over = SMOTE(sampling_strategy=o)
    under = RandomUnderSampler(sampling_strategy=u)
    steps = [('over', over), ('under', under), ('model', model)]
    pipeline = Pipeline(steps=steps)
    # evaluate pipeline
    scores = cross_validate(pipeline, X, y, scoring=['accuracy','f1','precision','recall','roc_auc'], cv=5, n_jobs=-1)
    #score = mean(scores)
    print(scores)
    print('SMOTE oversampling rate:%.1f, Random undersampling rate:%.1f ' % (o, u))

{'fit_time': array([47.58797121, 45.63895607, 45.86026192, 34.1026051 , 47.82134819]), 'score_time': array([0.127527  , 0.14071226, 0.14924169, 0.35685992, 0.13276386]), 'test_accuracy': array([0.77012168, 0.77772891, 0.78945825, 0.77721575, 0.77347702]), 'test_f1': array([0.33559322, 0.31803869, 0.28521653, 0.29766582, 0.32147563]), 'test_precision': array([0.36049158, 0.36746362, 0.38327759, 0.35698448, 0.36041359]), 'test_recall': array([0.31391201, 0.28033307, 0.22711058, 0.25525168, 0.2901308 ]), 'test_roc_auc': array([0.66328126, 0.66282205, 0.66477657, 0.66548734, 0.66649799])}
SMOTE oversampling rate:0.3, Random undersampling rate:0.7 


KeyboardInterrupt: 